In [2]:
BASE_URL = "https://huggingface.co/datasets/karpathy/fineweb-edu-100b-shuffle/resolve/main"

In [3]:
ids_to_download = list(range(1822))

In [5]:
import os
import requests

DATA_DIR = 'data'
os.makedirs(DATA_DIR, exist_ok=True)

filename = f'shard_{ids_to_download[0]:05d}.parquet'
filepath = os.path.join(DATA_DIR, filename)
if not os.path.exists(filepath):
    url = f'{BASE_URL}/{filename}'
    response = requests.get(url)
    temp_filepath = filepath + '.tmp'
    with open(temp_filepath, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    os.rename(temp_filepath, filepath)